<a href="https://colab.research.google.com/github/JasonD28/CS-175/blob/master/scripts/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pydicom

     |████████████████████████████████| 35.5MB 90kB/s 


In [3]:
%%shell
git clone https://github.com/JasonD28/CS-175.git
cd CS-175
cp model/dataset.py  ../

Cloning into 'CS-175'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 168 (delta 86), reused 47 (delta 18), pack-reused 0
Receiving objects: 100% (168/168), 1.43 MiB | 1.61 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [4]:
%%shell

git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.5.1

cp references/detection/utils.py ../
cp references/detection/engine.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 8389 (delta 0), reused 1 (delta 0), pack-reused 8384
Receiving objects: 100% (8389/8389), 10.25 MiB | 6.95 MiB/s, done.
Resolving deltas: 100% (5775/5775), done.
Branch 'v0.5.1' set up to track remote branch 'v0.5.1' from 'origin'.
Switched to a new branch 'v0.5.1'


In [0]:
import torch
from torch import optim
import torchvision
from dataset import PneumoniaDataset

import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.nn.functional import relu as Relu
from torch import sigmoid

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit

import utils

In [0]:
def Flatten(x):
    N, C, H, W = x.size() # read in N, C, H, W
    return x.view(N, -1)

In [0]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        ksize = 3
        netPadding = (ksize-1)//2
        self.convInput = nn.Conv2d(1, 32, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv32 = nn.Conv2d(32, 32, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv32to64 = nn.Conv2d(32, 64, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv64 = nn.Conv2d(64, 64, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv64to128 = nn.Conv2d(64, 128, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv128 = nn.Conv2d(128, 128, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv128to256 = nn.Conv2d(128, 256, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv256 = nn.Conv2d(256, 256, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv256to128 = nn.Conv2d(256, 128, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv128to64 = nn.Conv2d(128, 64, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv64to1 = nn.Conv2d(64, 1, kernel_size=ksize, stride=1, padding=netPadding)
        self.pooling = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pooling2 = nn.MaxPool2d(kernel_size=5, stride=2)
        self.upsampling = nn.Upsample(scale_factor=(2,2))
        self.linearto2048 = nn.Linear(4096,2048)
        self.linearto1 = nn.Linear(2048,1)

    def forward(self, x):
        # out = model(x) # Not sure what this does
        x = Relu(self.convInput(x), inplace=True)
        x = Relu(self.conv32(x), inplace=True)
        poolX = self.pooling(x)
        
        x2 = Relu(self.conv32to64(poolX), inplace=True)
        x2 = Relu(self.conv64(x2), inplace=True)
        poolX2 = self.pooling(x2)

        x3 = Relu(self.conv64(poolX2), inplace=True)
        x3 = Relu(self.conv64(x3), inplace=True)
        poolX3 = self.pooling(x3)
        
        x4 = Relu(self.conv64to128(poolX3), inplace=True)
        x4 = Relu(self.conv128(x4), inplace=True)
        poolX4 = self.pooling(x4)
        
        x5 = Relu(self.conv128to256(poolX4), inplace=True)
        
        up6 = self.upsampling(x5)
        x6 = Relu(self.conv256(up6), inplace=True)
        x6 = Relu(self.conv256(x6), inplace=True)
        poolX6 = self.pooling(x6)
        merge6 = poolX6 #torch.cat((x4,poolX6),-1)
        x6 = Relu(self.conv256(merge6), inplace=True)
        
        up7 = self.upsampling(x6)
        x7 = Relu(self.conv256(up7), inplace=True)
        x7 = Relu(self.conv256(x7), inplace=True)
        poolX7 = self.pooling(x7)
        merge7 = poolX7 #torch.cat((x3,poolX7),-1)
        x7 = Relu(self.conv256(merge7), inplace=True)
        
        up8 = self.upsampling(x7)
        x8 = Relu(self.conv256to128(up8), inplace=True)
        x8 = Relu(self.conv128(x8), inplace=True)
        poolX8 = self.pooling(x8)
        merge8 = poolX8 #torch.cat((x2,poolX8),-1)
        x8 = Relu(self.conv128(merge8), inplace=True)
        
        up9 = self.upsampling(x8)
        x9 = Relu(self.conv128to64(up9), inplace=True)
        x9 = Relu(self.conv64(x9), inplace=True)
        poolX9 = self.pooling(x9)
        merge9 = poolX9 #torch.cat((x1,poolX9),-1)
        x9 = Relu(self.conv64(merge9), inplace=True)
        
        x10 = Flatten(self.conv64to1(x9))
        x11 = Relu(self.linearto2048(x10), inplace=True)
        x12 = self.linearto1(x11)
        x13 = sigmoid(x12)
        # x10 = sigmoid(self.conv64to1(x9))
        
        return x13

In [0]:
root = "drive/My Drive/cs-175-data"

In [9]:
model = NeuralNet();
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

NeuralNet(
  (convInput): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv32): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv32to64): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv64): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv64to128): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv128): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv128to256): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv256): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv256to128): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv128to64): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv64to1): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [0]:
dataset = PneumoniaDataset(root, True)

In [0]:
test_dataset = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
test_dataset = torch.utils.data.Subset(test_dataset, indices[-50:])

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=0,
        collate_fn=utils.collate_fn)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)

optimizer = optim.Adam(model.parameters(), lr=1e-6)
lr_step = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

In [0]:
def train(train_data, dtype, model, loss_fn, optimizer, num_epochs=1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        #print(train_data.shape)
        for t, (image, target) in enumerate(train_data):
            #print(image)
            # images = list(img for img in image)
            imgs = torch.tensor([img.numpy() for img in image], dtype=torch.float32)
            x_var = Variable(imgs.type(dtype))
            l = []
            for tar in target:
              # print(len(tar['labels']))
              if len(tar['labels']) > 1:
                temp = [list(tar['labels'])[0]]
                l.append(temp)
              else:
                l.append(list(tar['labels']))
            # print(l)
            y_var = torch.tensor(l).cuda() # Variable(labels.type(dtype).cuda())
            # print(y_var)
            
            scores = model(imgs.cuda())
            # print(scores)
            loss = loss_fn(scores, y_var.float())
            if (t + 1) % 100 == 0:
              print('t = %d, loss = %.4f' % (t + 1, loss.item()))
              print(y_var)
              print(scores)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [0]:
gpu_dtype = torch.cuda.FloatTensor
loss_fn = nn.BCEWithLogitsLoss().type(gpu_dtype)

In [45]:
%%timeit
train(data_loader, gpu_dtype, model, loss_fn, optimizer, 1)

Starting epoch 1 / 1
t = 100, loss = -0.0409
tensor([[2],
        [2]], device='cuda:0')
tensor([[0.5109],
        [0.5109]], device='cuda:0', grad_fn=<SigmoidBackward>)
t = 200, loss = 0.4668
tensor([[0],
        [2]], device='cuda:0')
tensor([[0.5194],
        [0.5194]], device='cuda:0', grad_fn=<SigmoidBackward>)
t = 300, loss = 0.9893
tensor([[0],
        [0]], device='cuda:0')
tensor([[0.5243],
        [0.5243]], device='cuda:0', grad_fn=<SigmoidBackward>)
t = 400, loss = 0.7280
tensor([[0],
        [1]], device='cuda:0')
tensor([[0.5315],
        [0.5315]], device='cuda:0', grad_fn=<SigmoidBackward>)
t = 500, loss = 0.4587
tensor([[2],
        [0]], device='cuda:0')
tensor([[0.5413],
        [0.5413]], device='cuda:0', grad_fn=<SigmoidBackward>)
t = 600, loss = 0.1732
tensor([[1],
        [2]], device='cuda:0')
tensor([[0.5585],
        [0.5585]], device='cuda:0', grad_fn=<SigmoidBackward>)


KeyboardInterrupt: ignored

In [0]:
def check_accuracy(model, loader):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in loader:
        with torch.no_grad():
            x_var = Variable(x.type(gpu_dtype))

        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))